In [ ]:
# import the relevant libraries
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Using CDO, we have selected the grid boxes for iceland and the azores.

In [ ]:
# set the file paths for the data on JASMIN
azores_file_path="/home/users/benhutch/ERA5_psl/ERA5.azores-gridbox.psl.nc"
iceland_file_path="/home/users/benhutch/ERA5_psl/ERA5.iceland-gridbox.psl.nc"

Now we define a function to process the azores and iceland data

In [ ]:
# process the data
def process_data(file_path):
    # 1. Load data using xarray and dask
    ds = xr.open_dataset(file_path, chunks={'time': 10})

    # calculate the model mean state
    ds_mean = ds.mean(dim='time')

    # calculate the anomaly
    ds_anom = ds - ds_mean

return ds_anom

azores_anom = process_data(azores_file_path)
iceland_anom = process_data(iceland_file_path)

In [ ]:
# calculate the NAO index
# make sure to take the field mean over the gridboxes
azores_anom_mean = azores_anom.mean(dim=['lat','lon'])
iceland_anom_mean = iceland_anom.mean(dim=['lat','lon'])

# calculate the NAO index
nao_index = azores_anom_mean - iceland_anom_mean

# get the DJFM means
# take a rolling mean over 4 months
nao_index_roll = nao_index.rolling(time=4).mean()

In [ ]:
# Process both datasets
azores_gridbox_mean = process_data('ERA5.azores-gridbox.psl.nc')
iceland_gridbox_mean = process_data('ERA5.iceland-gridbox.psl.nc')

In [ ]:
# 6. Subtract the iceland seasonal gridbox means from the azores seasonal gridbox means to calculate the NAO anomaly
nao_anomaly = azores_gridbox_mean - iceland_gridbox_mean

# print the NAO anomaly
print(nao_anomaly)

In [ ]:
# 7. Take a forward running mean (e.g. the output for 1960 is the DJFM winters from 1961/62 to 68/69) for the whole dataset.
window_size = 8
nao_anomaly_rolling_mean = nao_anomaly.rolling(time=window_size, center=False).mean()

# Save the results to a netCDF file
nao_anomaly_rolling_mean.to_netcdf('NAO_anomaly_rolling_mean.nc')

In [ ]:
# plot the NAO anomaly
# with the years on the x-axis
# and the NAO anomaly on the y-axis
nao_anomaly_rolling_mean.plot()
plt.show()